# Purpose

**2023-02-21: v0.6.0** update
<br>For some reason the previous table with CAU embeddings was deleted. I tried to run this notebook to re-upload the embeddings for v0.6.1.

However, the authentication in vertex AI notebook is broken, so I can't run the function to upload the data from here.

**See the colab notebook that loads the data using SQL instead.**

--- 

**2022-08-30: v0.6.0**
The default parquet embedding format for my embeddings (1 row per column) is not favored for bigquery & other steam standards.

The preferred format is: 1 column that has repeated records. For example:
- `data-prod-165221.ml_content.subreddit_embeddings_ft2` 
    - [console link](https://console.cloud.google.com/bigquery?project=data-science-prod-218515&ws=!1m10!1m4!4m3!1sdata-prod-165221!2sml_content!3ssimilar_subreddit_ft2!1m4!4m3!1sdata-prod-165221!2sml_content!3ssubreddit_embeddings_ft2)
    - github link
        - https://github.snooguts.net/reddit/gazette-models/blob/master/similar_subreddit/embeddings/__main__.py#L105-L112

In this notebook we convert a dataframe into a new-line delimited JSON file. 
<br>With pandas we can vectorize this function instead of having to loop through each row (subreddit) individually.

In this notebook we're loading 2 separate embedding flavors based on the `embeddings_artifact_path` and loading to the BQ in two separate partitions.

---

```python
sr_embedding_dict = {
    "pt": date_today,
    "model_name": MODEL_NAME,
    "model_version": MODEL_VERSION,
    "subreddit_id": sr_dict["subreddit_id"],
    "subreddit_name": subreddit_name_lowercase,
    "embedding": sr_embedding.tolist(),
}
```

In BQ:
```
Field name		Type	Mode		Description
model_name  	STRING	NULLABLE		Model name	
model_version	STRING	NULLABLE		Model version	
subreddit_id	STRING	NULLABLE		Subreddit id	
subreddit_name	STRING	NULLABLE		Lower case subreddit name	
**embedding		FLOAT	REPEATED		Subreddit embeddings
```

# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import os
import logging
from logging import info
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import mlflow
import hydra

import subclu
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.utils import set_working_directory, get_project_subfolder
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric,
    elapsed_time,
)

from subclu.utils.mlflow_logger import MlflowLogger
from subclu.models.bq_embedding_schemas import embeddings_schema
from subclu.models.reshape_embeddings_for_bq import reshape_embeddings_to_ndjson, reshape_embeddings_and_upload_to_bq
from subclu.utils.big_query_utils import load_data_to_bq_table

print_lib_versions([hydra, mlflow, np, pd, subclu])

python		v 3.7.10
===
hydra		v: 1.1.0
mlflow		v: 1.16.0
numpy		v: 1.19.5
pandas		v: 1.2.4
subclu		v: 0.6.1


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Load configs for embeddings to reshape & upload to BQ

The embedding aggregation should've been logged to `mlflow` so we should be able to
- make calls to mlflow to get the embeddings
- add the new embeddings format to the original job

---
We have 2 strategies for getting subreddit embeddings. For now we'll only make available the `extra weight to subreddit meta` because that performed slightly better in previous taxonomy/rating models.




In [4]:
cfg_reshape_embeddings = LoadHydraConfig(
    config_name='reshape_embeddings_for_bq-subreddit-v0.6.1.yaml',
    config_path="../config",
)

In [25]:
cfg_reshape_embeddings_wt = LoadHydraConfig(
    config_name='reshape_embeddings_for_bq-subreddit-v0.6.1_desc_extra_weight.yaml',
    config_path="../config",
)

In [5]:
for k_, v_ in cfg_reshape_embeddings.config_dict.items():
    if isinstance(v_, dict):
        print(f"{k_}:")
        for k2_, v2_ in v_.items():
            pass
            print(f"    {k2_}: {v2_}")
    else:
        print(f"{k_}: {v_}")

data_text_and_metadata:
    dataset_name: v0.6.1 inputs. ~110k seed subreddits, ~340k with 3+ posts, ~700k total subreddits
    bucket_name: i18n-subreddit-clustering
    folder_subreddits_text_and_meta: i18n_topic_model_batch/runs/20221107/subreddits_fix/text
    folder_subreddits_text_and_meta_crowd_topic: i18n_topic_model_batch/runs/20221107/subreddits/text
    folder_posts_text_and_meta: i18n_topic_model_batch/runs/20221107/posts
    folder_comments_text_and_meta: i18n_topic_model_batch/runs/20221107/comments
    folder_post_and_comment_text_and_meta: i18n_topic_model_batch/runs/20221107/post_and_comment_text_combined/text_all
data_embeddings_to_aggregate:
    bucket_embeddings: i18n-subreddit-clustering
    post_and_comments_folder_embeddings: i18n_topic_model_batch/runs/20221107/post_and_comment_text_combined/text_all/embedding/2022-11-07_081017
    subreddit_desc_folder_embeddings: i18n_topic_model_batch/runs/20221107/subreddits/text/embedding/2022-11-07_074632
    col_subreddit

In [26]:
for k_, v_ in cfg_reshape_embeddings_wt.config_dict.items():
    if isinstance(v_, dict):
        print(f"{k_}:")
        for k2_, v2_ in v_.items():
            pass
            print(f"    {k2_}: {v2_}")
    else:
        print(f"{k_}: {v_}")

data_text_and_metadata:
    dataset_name: v0.6.1 inputs. ~110k seed subreddits, ~340k with 3+ posts, ~700k total subreddits
    bucket_name: i18n-subreddit-clustering
    folder_subreddits_text_and_meta: i18n_topic_model_batch/runs/20221107/subreddits_fix/text
    folder_subreddits_text_and_meta_crowd_topic: i18n_topic_model_batch/runs/20221107/subreddits/text
    folder_posts_text_and_meta: i18n_topic_model_batch/runs/20221107/posts
    folder_comments_text_and_meta: i18n_topic_model_batch/runs/20221107/comments
    folder_post_and_comment_text_and_meta: i18n_topic_model_batch/runs/20221107/post_and_comment_text_combined/text_all
data_embeddings_to_aggregate:
    bucket_embeddings: i18n-subreddit-clustering
    post_and_comments_folder_embeddings: i18n_topic_model_batch/runs/20221107/post_and_comment_text_combined/text_all/embedding/2022-11-07_081017
    subreddit_desc_folder_embeddings: i18n_topic_model_batch/runs/20221107/subreddits/text/embedding/2022-11-07_074632
    col_subreddit

# Start MLflow & Get Artifact Paths

This mlflow server will help us get the artifacts & subreddit-embeddings for the `mlflow-uuid` flagged in the configurations below.

In [6]:
mlf = MlflowLogger(tracking_uri=cfg_reshape_embeddings.config_dict['mlflow_tracking_uri'])

In [7]:
mlf.list_experiment_meta(output_format='pandas').tail(9)

,experiment_id,name,artifact_location,lifecycle_stage
35,35,v0.6.0_mUSE_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/35,active
36,36,v0.6.0_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/36,active
37,37,v0.6.0_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/37,active
38,38,v0.6.0_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/38,active
39,39,v0.6.1_mUSE_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/39,active
40,40,v0.6.1_mUSE_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/40,active
41,41,v0.6.1_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/41,active
42,42,v0.6.1_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/42,active
43,43,v0.6.1_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/43,active


In [8]:
%%time

df_mlf = mlf.search_all_runs(experiment_ids=[40])
print(df_mlf.shape)

(4, 43)
CPU times: user 68.4 ms, sys: 55.2 ms, total: 124 ms
Wall time: 2.6 s


In [9]:
# only show finished runs
df_mlf[df_mlf['status'] == "FINISHED"].iloc[:5, :13]

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.df_v_subs-rows,metrics.df_v_post_comments-rows,metrics.df_v_post_comments-cols,metrics.time_fxn-data_loading_time,metrics.cpu_count,metrics.memory_free,metrics.memory_used
2,91ac7ca171024c779c0992f59470c81b,40,FINISHED,gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts,2022-11-07 21:38:57.662000+00:00,2022-11-23 02:06:49.677000+00:00,781653.0,53597817.0,515.0,5.742884,96.0,1283359.0,798566.0


## Get path to latest reshaped data
Since we already reshaped the data and it's in GCS, let's read that instead of reshaping & uploading to GCS

In [56]:
l_artifacts_top_level = mlf.list_run_artifacts(
    run_id=cfg_reshape_embeddings_wt.config_dict['mlflow_run_id'],
    only_top_level=True,
    verbose=True,
    full_path=True,
)
l_artifacts_all = mlf.list_run_artifacts(
    run_id=cfg_reshape_embeddings_wt.config_dict['mlflow_run_id'],
    only_top_level=False,
    verbose=False,
    full_path=True,
)

22:39:53 | INFO | "   342 <- Artifacts to check count"
22:39:53 | INFO | "   342 <- Artifacts clean count"
22:39:53 | INFO | "     9 <- Artifacts & folders at TOP LEVEL clean count"
22:39:59 | INFO | "   342 <- Artifacts clean count"
22:39:59 | INFO | "     9 <- Artifacts & folders at TOP LEVEL clean count"


In [57]:
l_artifacts_top_level

['gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/ann_df-2022-11-22_185903',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/ann_df-239774-2022-11-22_230904',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/ann_ndjson-239774-2022-11-22_230904',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/ann_ndjson-239774-2022-11-22_230904_fix',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_posts_agg_c1',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1_ndjson',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1_unweighted',
 'gs://i18n-subreddit-cluste

In [60]:
# get path for parquet file. This should be a full folder with multiple parquet files
path_parquet_ = (
    cfg_reshape_embeddings.config_dict['embeddings_artifact_path']
)

folder_parquet_ = (
    cfg_reshape_embeddings_wt.config_dict['embeddings_artifact_path']
)
folder_parquet_full_ = [i_ for i_ in l_artifacts_top_level if i_.split('/')[-1] == folder_parquet_][0]

print(f"Folder with parquet files:\n{path_parquet_}")
print(f"\nFolder with parquet files (full):\n{folder_parquet_full_}\n")

[_ for _ in l_artifacts_all if folder_parquet_ == _.split('/')[-2]]

Folder with parquet files:
df_subs_agg_c1

Folder with parquet files (full):
gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1



['gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1/_common_metadata',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1/_metadata',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1/part.0.parquet',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1/part.1.parquet',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1/part.2.parquet',
 'gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1/part.3.parquet']

In [52]:
# get path for latest ndjson output FILE
#  NOTE: there could be multiple runs of this file
path_ndjson_ = (
    cfg_reshape_embeddings_wt.config_dict['embeddings_artifact_path'] + '_ndjson'
)
l_ndjson_files_ = [_ for _ in l_artifacts_all if path_ndjson_ in _]
ndjson_path_ = l_ndjson_files_[-1]
print(f"ndjson Files:\n{l_ndjson_files_}")

print(f"\nFile to upload to BQ:\n{ndjson_path_}")

ndjson Files:
['gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1_ndjson/subreddit_embeddings_2022-11-18_171217.json']

File to upload to BQ:
gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1_ndjson/subreddit_embeddings_2022-11-18_171217.json


# Run new function to only upload existing reshaped data

In [63]:
%%time

info(f"Creating table from file:\n{ndjson_path_}")

load_data_to_bq_table(
    uri=ndjson_path_,
    bq_project=cfg_reshape_embeddings_wt.config_dict['bq_project'],
    bq_dataset=cfg_reshape_embeddings_wt.config_dict['bq_dataset'],
    bq_table_name=cfg_reshape_embeddings_wt.config_dict['bq_table'],
    schema=embeddings_schema(),
    partition_column='pt',
    table_description=cfg_reshape_embeddings_wt.config_dict['bq_table_description'],
    update_table_description=False,
)

22:50:33 | INFO | "Creating table from file:
gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1_ndjson/subreddit_embeddings_2022-11-18_171217.json"
22:50:35 | INFO | "Loading data to table:
  reddit-employee-datasets.david_bermejo.cau_subreddit_embeddings"
22:50:35 | INFO | "Table reddit-employee-datasets.david_bermejo.cau_subreddit_embeddings already exist"
22:50:35 | INFO | "  0 rows in table BEFORE adding data"
22:51:05 | INFO | "  0 rows in table AFTER adding data"


CPU times: user 59.5 ms, sys: 68 ms, total: 128 ms
Wall time: 31.3 s


## Load SUBREDDIT embeddings from parquet files [for display]

This helps us sample what the parquet embeddings look like (columns & shape).
ETA:
- ~2 minutes: download from GCS to local cache
- ~30 seconds: loading into pandas (after download to local)

In [27]:
%%time

df_agg_sub_c2 = mlf.read_run_artifact(
    run_id=cfg_reshape_embeddings_wt.config_dict['mlflow_run_id'],
    artifact_folder=cfg_reshape_embeddings_wt.config_dict['embeddings_artifact_path'],
    read_function='pd_parquet',
    verbose=False,
)
print(df_agg_sub_c2.shape)

22:10:39 | INFO | "Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1"
100%|########################################| 14/14 [00:00<00:00, 31317.47it/s]
22:10:39 | INFO | "  Parquet files found:     4"
22:10:39 | INFO | "  Parquet files to use:     4"


(781653, 515)
CPU times: user 10.1 s, sys: 5.7 s, total: 15.8 s
Wall time: 28 s


In [10]:
%%time

df_agg_sub_c = mlf.read_run_artifact(
    run_id=cfg_reshape_embeddings.config_dict['mlflow_run_id'],
    artifact_folder=cfg_reshape_embeddings.config_dict['embeddings_artifact_path'],
    read_function='pd_parquet',
    verbose=False,
)
print(df_agg_sub_c.shape)

19:54:36 | INFO | "Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1_unweighted"
100%|##########################################| 7/7 [00:00<00:00, 17149.61it/s]
19:54:36 | INFO | "  Parquet files found:     4"
19:54:36 | INFO | "  Parquet files to use:     4"


(781653, 515)
CPU times: user 9.27 s, sys: 7.03 s, total: 16.3 s
Wall time: 29.1 s


In [14]:
df_agg_sub_c.iloc[:5, :10]

,subreddit_id,subreddit_name,posts_for_embeddings_count,embeddings_0,embeddings_1,embeddings_2,embeddings_3,embeddings_4,embeddings_5,embeddings_6
0,t5_1001tl,jewel_xo,1,-0.041036,-0.008650,-0.013949,0.034232,-0.045981,0.021885,0.024000
1,t5_1004au,tisbutafleshwound,3,0.010298,-0.000277,-0.004013,0.017620,-0.076757,0.031346,0.069366
2,t5_1006a0,sethigh,1,0.028022,0.007011,-0.005661,0.025936,-0.007770,0.031322,0.056113
3,t5_1008xr,asiandiasporamusic,2,0.016526,-0.006581,0.017150,0.007448,0.029106,0.041690,0.038882
4,t5_1009a3,memesenespanol,299,-0.005113,-0.005898,-0.012267,0.006103,-0.009704,0.039449,0.010752


In [30]:
df_agg_sub_c2.iloc[:5, :10]

,subreddit_id,subreddit_name,posts_for_embeddings_count,embeddings_0,embeddings_1,embeddings_2,embeddings_3,embeddings_4,embeddings_5,embeddings_6
0,t5_1001tl,jewel_xo,1,-0.028712,-0.027187,0.024826,0.046359,0.006391,0.046740,-0.005454
1,t5_1004au,tisbutafleshwound,3,0.010298,-0.000277,-0.004013,0.017620,-0.076757,0.031346,0.069366
2,t5_1006a0,sethigh,1,0.027356,0.032256,-0.022585,-0.004125,0.013944,0.039986,-0.007854
3,t5_1008xr,asiandiasporamusic,2,-0.011276,0.000720,-0.010621,0.021452,0.035787,0.040039,0.034074
4,t5_1009a3,memesenespanol,299,-0.005113,-0.005898,-0.012267,0.006103,-0.009704,0.039449,0.010752


Check whether embeddings values are the same for subreddits with `3>= posts`

In [40]:
n_rows_ = 6
n_cols_ = 10

display(
    df_agg_sub_c2.iloc[:n_rows_, :3].merge(
        pd.DataFrame(
            np.isclose(
                df_agg_sub_c.iloc[:n_rows_, 3:n_cols_], df_agg_sub_c2.iloc[:n_rows_, 3:n_cols_]
            ),
            columns=df_agg_sub_c2.iloc[:n_rows_, 3:n_cols_].columns,
        ),
        how='left',
        left_index=True,
        right_index=True,
    )
)
del n_rows_, n_cols_

,subreddit_id,subreddit_name,posts_for_embeddings_count,embeddings_0,embeddings_1,embeddings_2,embeddings_3,embeddings_4,embeddings_5,embeddings_6
0,t5_1001tl,jewel_xo,1,False,False,False,False,False,False,False
1,t5_1004au,tisbutafleshwound,3,True,True,True,True,True,True,True
2,t5_1006a0,sethigh,1,False,False,False,False,False,False,False
3,t5_1008xr,asiandiasporamusic,2,False,False,False,False,False,False,False
4,t5_1009a3,memesenespanol,299,True,True,True,True,True,True,True
5,t5_100a1y,karstcast,7,True,True,True,True,True,True,True


## Check distribution of posts for embeddings
We'd expect ~340k subs with 3+ posts

In [41]:
df_agg_sub_c2['posts_for_embeddings_count'].describe()

count    781653.000000
mean         68.569835
std         487.025214
min           0.000000
25%           1.000000
50%           2.000000
75%           8.000000
max        8400.000000
Name: posts_for_embeddings_count, dtype: float64

In [45]:
assert np.allclose(
    df_agg_sub_c['posts_for_embeddings_count'].describe(), 
    df_agg_sub_c2['posts_for_embeddings_count'].describe()
)

df_agg_sub_c['posts_for_embeddings_count'].describe()

count    781653.000000
mean         68.569835
std         487.025214
min           0.000000
25%           1.000000
50%           2.000000
75%           8.000000
max        8400.000000
Name: posts_for_embeddings_count, dtype: float64

In [42]:
value_counts_and_pcts(
    pd.cut(
        df_agg_sub_c2['posts_for_embeddings_count'],
        bins=[-1, 0, 1, 2, 3, 4, 5, np.inf],
        labels=["0 posts", "1 post", '2 posts', '3 posts', '4 posts', '5 posts', '6+ posts']
    ),
    sort_index=True,
    sort_index_ascending=False,
    cumsum_count=True,
)

,posts_for_embeddings_count-count,posts_for_embeddings_count-percent,posts_for_embeddings_count-cumulative_sum,posts_for_embeddings_count-pct_cumulative_sum
6+ posts,"232,401",29.7%,"232,401",29.7%
5 posts,"23,732",3.0%,"256,133",32.8%
4 posts,"33,723",4.3%,"289,856",37.1%
3 posts,"57,946",7.4%,"347,802",44.5%
2 posts,"128,068",16.4%,"475,870",60.9%
1 post,"235,794",30.2%,"711,664",91.0%
0 posts,"69,989",9.0%,"781,653",100.0%


In [43]:
value_counts_and_pcts(
    pd.cut(
        df_agg_sub_c['posts_for_embeddings_count'],
        bins=[-1, 0, 1, 2, 3, 4, 5, np.inf],
        labels=["0 posts", "1 post", '2 posts', '3 posts', '4 posts', '5 posts', '6+ posts']
    ),
    sort_index=True,
    sort_index_ascending=False,
    cumsum_count=True,
)

,posts_for_embeddings_count-count,posts_for_embeddings_count-percent,posts_for_embeddings_count-cumulative_sum,posts_for_embeddings_count-pct_cumulative_sum
6+ posts,"232,401",29.7%,"232,401",29.7%
5 posts,"23,732",3.0%,"256,133",32.8%
4 posts,"33,723",4.3%,"289,856",37.1%
3 posts,"57,946",7.4%,"347,802",44.5%
2 posts,"128,068",16.4%,"475,870",60.9%
1 post,"235,794",30.2%,"711,664",91.0%
0 posts,"69,989",9.0%,"781,653",100.0%
